In [1]:
from kaggle_environments import make
from scipy.stats import norm

import hyperopt
import numpy as np
import os
if not os.path.exists('tmp'):
    os.mkdir('tmp')
    
base_path = '/Users/sergmiller/Documents/my/bandits'

Loading environment football failed: No module named 'gfootball'


In [2]:
from utils.bandits import Agent, init_agent, \
    compare, gittins_bb_delta, init_template, neural, gittins_with_random, bench

In [10]:
from importlib import reload
import utils
reload(utils)
reload(utils.bandits)
from utils.bandits import exact_gittins

In [4]:
compare(Agent(text=utils.bandits.exact_gittins), Agent(text=gittins_with_random))

(1.4968878586693142e-05,
 42.4,
 30.971599894096528,
 array([687., 657., 613., 639., 716., 613., 606., 654., 619., 591.]),
 array([612., 592., 529., 616., 688., 623., 561., 614., 544., 592.]))

In [5]:
bench(Agent(text=utils.bandits.exact_gittins))

['gittins', (4.2410388356091665e-07, 45.6, 28.510348998214667)]
['gittins_with_random', (0.0018946078240590072, 33.7, 34.30757933751666)]
['gittins_with_count_my', (0.5443322947442247, 5.3, 27.64434842784326)]
['gittins_with_count_rival_drift', (2.069588791130939e-06, 28.8, 19.18749592833833)]
['gittins_with_my_and_count_rival_drift', (0.8731023219367382, 1.0, 19.79898987322333)]
['gittins_bb', (0.00027884018531413636, 33.2, 28.88875213642846)]
['gittins_bb_delta', (0.7698044398391066, 3.6, 38.902956185873585)]
['softmax_ucb', (7.928191572141328e-06, 37.3, 26.404734423962683)]
['multiarmed_bandit_agent', (0.13655970869685635, 10.5, 22.303587155433092)]
['upper_confidence', (4.963109977336576e-61, 147.8, 28.357714999625763)]
['ucb_decay', (7.461902643596642e-07, 39.3, 25.111949346874685)]
['bayesian_ucb', (0.6950011664656811, -3.3, 26.615972648017205)]
['thompson', (0.25003483765841505, -10.8, 29.691076100404306)]
['neural', (0.237276505928668, -9.6, 25.687350972803717)]
<string>:82: Ru

[['gittins', (4.2410388356091665e-07, 45.6, 28.510348998214667)],
 ['gittins_with_random', (0.0018946078240590072, 33.7, 34.30757933751666)],
 ['gittins_with_count_my', (0.5443322947442247, 5.3, 27.64434842784326)],
 ['gittins_with_count_rival_drift',
  (2.069588791130939e-06, 28.8, 19.18749592833833)],
 ['gittins_with_my_and_count_rival_drift',
  (0.8731023219367382, 1.0, 19.79898987322333)],
 ['gittins_bb', (0.00027884018531413636, 33.2, 28.88875213642846)],
 ['gittins_bb_delta', (0.7698044398391066, 3.6, 38.902956185873585)],
 ['softmax_ucb', (7.928191572141328e-06, 37.3, 26.404734423962683)],
 ['multiarmed_bandit_agent', (0.13655970869685635, 10.5, 22.303587155433092)],
 ['upper_confidence', (4.963109977336576e-61, 147.8, 28.357714999625763)],
 ['ucb_decay', (7.461902643596642e-07, 39.3, 25.111949346874685)],
 ['bayesian_ucb', (0.6950011664656811, -3.3, 26.615972648017205)],
 ['thompson', (0.25003483765841505, -10.8, 29.691076100404306)],
 ['neural', (0.237276505928668, -9.6, 25.68

In [7]:
compare(
    Agent(text=utils.bandits.exact_gittins_with_my_and_count_rival_drift.format("decay=0.03")),
    Agent(file='/Users/sergmiller/Documents/my/bandits/kernels/thompson.py')
)

(0.07719266577147046,
 -15.1,
 27.020177645604036,
 array([645., 658., 621., 682., 519., 656., 677., 669., 608., 678.]),
 array([644., 643., 675., 652., 576., 664., 717., 673., 629., 691.]))

In [8]:
compare(
    Agent(text=utils.bandits.exact_gittins_with_my_and_count_rival_drift.format("decay=0.03")),
    Agent(text=neural)
)

(0.3760897672331939,
 9.4,
 33.583329197683774,
 array([599., 605., 599., 611., 630., 703., 605., 666., 683., 571.]),
 array([600., 581., 608., 654., 635., 619., 610., 643., 641., 587.]))

In [9]:
def f(input):
    print(input)
    exact_gittins_custom = init_template(utils.bandits.exact_gittins_with_my_and_count_rival_drift, input)
    res = compare(Agent(text=utils.bandits.exact_gittins), Agent(text=exact_gittins_custom), T=10)[:3]
    print(res[1] + res[2], res)
    return res[1] + res[2]  # since res[1] < 0

In [14]:
search_space = {
  "alpha": hyperopt.hp.uniform('alpha', 0.05, 0.95),
  "distortion_horizon": hyperopt.hp.uniform('distortion_horizon', 1.05, 10),
  "p": hyperopt.hp.uniform('p', 0.05, 0.95),
  "q": hyperopt.hp.uniform('q', 0.05, 0.95),
  "eps": hyperopt.hp.uniform('eps', 0, 1e-1),
  "rival_drift": hyperopt.hp.normal('rival_drift', -1e-3, 1e-3),
}

# trials = hyperopt.Trials()
best = hyperopt.fmin(
  fn=f,
  space=search_space,
  algo=hyperopt.tpe.suggest,
  max_evals=400,
  verbose=True,
  trials=trials)
print(best)



{'alpha': 0.327577676428244, 'distortion_horizon': 3.879261305736294, 'eps': 0.028575954636710012, 'p': 0.88266086119104, 'q': 0.6670363553220694, 'rival_drift': -0.0003661796251315407}
27.049011281025138                                       
(0.9525096856129976, 0.5, 26.549011281025138)            
{'alpha': 0.1063059276316673, 'distortion_horizon': 2.639560096248874, 'eps': 0.04070996045266857, 'p': 0.6917154876557291, 'q': 0.5783025261142943, 'rival_drift': -0.000723411924183151}
45.12066088834041                                                                  
(0.014260278374446362, 19.7, 25.420660888340414)                                   
{'alpha': 0.35384433588073994, 'distortion_horizon': 8.017275395960773, 'eps': 0.03699691011199045, 'p': 0.5897365423487383, 'q': 0.4769531643506638, 'rival_drift': -0.0015535799028164343}
22.753540961133073                                                                 
(0.9042277912861212, -0.9, 23.65354096113307)                         

KeyboardInterrupt: 

In [17]:
bb_delta_exact_gittins_params = {'alpha': 0.4685023287045787,
                                 'distortion_horizon': 1.3854054259517206,
                                 'eps': 0.031610535865854904, 
                                 'p': 0.519036315706196,
                                 'q': 0.8842598985284099,
                                 'rival_drift': 0.0035039229431558017}
-2.5916690765377                                                                  
# (0.00019065620309177938, -17.0, 14.4083309234623)  

-2.5916690765377

In [18]:
exact_gittins_bb_delta = init_template(
    utils.bandits.exact_gittins_with_my_and_count_rival_drift, bb_delta_exact_gittins_params)

In [29]:
compare(Agent(text=utils.bandits.neural), Agent(text=exact_gittins_bb_delta))

(0.3125083445907548,
 -6.6,
 20.66494616494318,
 array([661., 554., 643., 658., 714., 629., 557., 534., 630., 700.]),
 array([644., 546., 654., 630., 735., 650., 591., 557., 655., 684.]))

In [6]:
compare(
    Agent(text=utils.bandits.neural.format("use_reinforce=True", "")),
    Agent(text=utils.bandits.neural)
)

(0.0,
 -503.5,
 25.429313793337013,
 array([213., 240., 249., 264., 241., 304., 262., 240., 282., 274.]),
 array([695., 714., 746., 787., 793., 796., 779., 775., 757., 762.]))

In [33]:
compare(
    Agent(text=utils.bandits.neural.format("use_reinforce=True", "self.policy_baseline=exact_gittins")),
    Agent(text=utils.bandits.neural)
)

(4.599645879414189e-132,
 -83.2,
 10.759182125050213,
 array([127., 144., 128., 123., 127., 169., 126., 136., 118., 166.]),
 array([212., 238., 237., 196., 211., 247., 198., 221., 192., 244.]))

In [7]:
compare(
    Agent(text=utils.bandits.neural.format("use_reinforce=True", "self.policy_baseline=gittins")),
    Agent(text=utils.bandits.neural)
)

(3.4594435551496266e-69,
 -513.6,
 92.38203288518824,
 array([185., 185., 231., 328., 215., 317., 266., 177., 404., 195.]),
 array([802., 793., 798., 703., 759., 744., 728., 813., 785., 714.]))

In [8]:
compare(
    Agent(text=utils.bandits.neural.format("use_reinforce=True", "self.policy_baseline=thompson")),
    Agent(text=utils.bandits.neural)
)

(2.0562532984152424e-24,
 -334.6,
 103.77109424112284,
 array([299., 328., 444., 449., 303., 287., 441., 234., 445., 304.]),
 array([693., 694., 698., 723., 630., 707., 616., 745., 639., 735.]))

In [9]:
compare(
    Agent(text=utils.bandits.neural.format("use_reinforce=True", "self.policy_baseline=bucb")),
    Agent(text=utils.bandits.neural)
)

(2.097823765748286e-268,
 -483.2,
 43.655011167104284,
 array([292., 309., 300., 235., 212., 241., 228., 246., 287., 232.]),
 array([769., 770., 747., 746., 716., 718., 649., 785., 719., 795.]))

In [13]:
#batch_size=1
compare(
    Agent(text=utils.bandits.neural.format("use_reinforce=True", "self.policy_baseline=exact_gittins")),
    Agent(text=utils.bandits.neural)
)

(1.497225204072075e-207,
 -266.9,
 27.4534150881088,
 array([452., 391., 455., 476., 476., 396., 462., 382., 401., 492.]),
 array([737., 687., 742., 705., 760., 660., 696., 671., 684., 710.]))

In [17]:
compare(
    Agent(text=utils.bandits.neural.format("use_reinforce,batch_size=True,4", "self.policy_baseline=exact_gittins")),
    Agent(text=utils.bandits.neural)
)

(2.8737658636270172e-64,
 -185.2,
 34.6,
 array([480., 542., 492., 421., 530., 496., 491., 463., 530., 468.]),
 array([673., 688., 663., 665., 663., 663., 723., 656., 690., 681.]))

In [14]:
compare(
    Agent(text=utils.bandits.neural.format("use_reinforce,lr=True,1e-4", "self.policy_baseline=exact_gittins")),
    Agent(text=utils.bandits.neural)
)

(9.94498485075708e-19,
 -62.2,
 22.261176967986216,
 array([598., 559., 582., 606., 650., 563., 551., 605., 586., 598.]),
 array([646., 634., 626., 714., 675., 630., 638., 663., 638., 656.]))

In [18]:
compare(
    Agent(text=utils.bandits.neural.format("use_reinforce,lr,batch_size=True,1e-4,4", "self.policy_baseline=exact_gittins")),
    Agent(text=utils.bandits.neural)
)

(0.021258942115341867,
 -21.8,
 29.929249907072514,
 array([637., 575., 610., 709., 667., 621., 534., 649., 582., 553.]),
 array([693., 599., 631., 686., 723., 622., 567., 620., 601., 613.]))

In [21]:
def f_neural(input):
    print(input)
    input["use_reinforce"] = True
    baseline = input["baseline"]
    input.pop("baseline")
    neural_custom = init_template(
        utils.bandits.neural.format("{}","self.policy_baseline={}".format(baseline)),
        input)
    res = compare(Agent(text=utils.bandits.neural), Agent(text=neural_custom), T=10)[:3]
    f_out = lambda mu, sigma: (mu + sigma) * ( 1 + (mu >= 0)) # since expect mu < 0
    score = f_out(res[1], res[2])
    print(score, res)
    return score

In [22]:
search_space = {
    "lr": hyperopt.hp.loguniform('lr', np.log(1e-5), np.log(1e-1)),
    "batch_size": hyperopt.hp.randint("batch_size", 1, 32),
    "baseline" : hyperopt.hp.choice("baseline", ["exact_gittins", "gittins", "mu", "bucb", "thompson"])
}

trials = hyperopt.Trials()
best = hyperopt.fmin(
  fn=f_neural,
  space=search_space,
  algo=hyperopt.tpe.suggest,
  max_evals=500,
  verbose=True,
  trials=trials)
print(best)



{'baseline': 'thompson', 'batch_size': 21, 'lr': 0.0016757988938789416}
464.3316199171167                                      
(5.569016160658724e-68, 196.5, 35.66580995855835)      
{'baseline': 'exact_gittins', 'batch_size': 26, 'lr': 0.001685708390030079}        
244.53193861793318                                                                 
(2.9381019513427697e-19, 90.4, 31.865969308966584)                                 
{'baseline': 'bucb', 'batch_size': 5, 'lr': 0.002277444805136519}                   
977.7320801894318                                                                   
(0.0, 453.4, 35.466040094715964)                                                    
{'baseline': 'gittins', 'batch_size': 15, 'lr': 7.255328699759645e-05}              
12.78952592391964                                                                   
(0.29665788478268407, -6.3, 19.08952592391964)                                      
{'baseline': 'exact_gittins', 'batch_size': 26, 'lr': 

KeyboardInterrupt: 

In [ ]:
{'baseline': 'gittins', 'batch_size': 22, 'lr': 0.0005695633314932441}                  
6.208483757967439                                                                       
(0.010243694442692552, -26.8, 33.00848375796744)

{'baseline': 'gittins', 'batch_size': 31, 'lr': 0.00012963507094867122}               
-0.2778251988699729                                                                   
(0.0013081063920359502, -17.2, 16.922174801130026)

In [23]:
compare(
    Agent(text=utils.bandits.neural.format("use_reinforce,lr,batch_size=True,0.00013,31", "self.policy_baseline=gittins")),
    Agent(text=utils.bandits.neural)
)

(0.49817728892809543,
 -5.9,
 27.544327909753036,
 array([663., 580., 618., 668., 653., 630., 571., 572., 635., 641.]),
 array([633., 620., 576., 705., 637., 620., 587., 608., 648., 656.]))

In [24]:
compare(
    Agent(text=utils.bandits.neural.format("use_reinforce,lr,batch_size=True,0.00013,1", "self.policy_baseline=gittins")),
    Agent(text=utils.bandits.neural)
)

(0.06183377906566946,
 11.1,
 18.796010214936572,
 array([630., 682., 679., 653., 649., 604., 633., 613., 575., 629.]),
 array([618., 663., 668., 627., 635., 607., 580., 615., 573., 650.]))

In [25]:
a = Agent(text=utils.bandits.neural.format("use_reinforce,lr,batch_size=True,0.00013,1", "self.policy_baseline=gittins"))
init_agent(a)
a.file

'tmp/b_0.293338061860546.py'

In [26]:
bench(a)

['gittins', (0.000605194887147763, 31.2, 28.77081854935657)]
['gittins_with_random', (0.042883140092197125, 23.8, 37.16934220564039)]
['gittins_with_count_my', (0.06594355279157432, -12.8, 22.01272359341297)]
['gittins_with_count_rival_drift', (0.002452870138590082, 30.5, 31.841011290472544)]
['gittins_with_my_and_count_rival_drift', (0.4324989667265736, -6.3, 25.38129232328409)]
['gittins_bb', (0.39709422435313446, 6.6, 24.646297896438725)]
['gittins_bb_delta', (0.9625877451708813, 0.5, 33.70830758136635)]
['softmax_ucb', (0.00018081513927886215, 35.7, 30.14979270243827)]
['multiarmed_bandit_agent', (0.294236542913377, 7.0, 21.104501889407388)]
['upper_confidence', (6.578580142975601e-53, 142.1, 29.35114989229553)]
['ucb_decay', (0.0005915179478898581, 32.4, 29.823480682173905)]
['bayesian_ucb', (0.2697398047344016, -8.8, 25.21428166734083)]
['thompson', (0.9482518422547107, -0.4, 19.48948434412773)]
['neural', (0.47011626999402834, 7.4, 32.39814809522298)]
<string>:82: RuntimeWarning

[['gittins', (0.000605194887147763, 31.2, 28.77081854935657)],
 ['gittins_with_random', (0.042883140092197125, 23.8, 37.16934220564039)],
 ['gittins_with_count_my', (0.06594355279157432, -12.8, 22.01272359341297)],
 ['gittins_with_count_rival_drift',
  (0.002452870138590082, 30.5, 31.841011290472544)],
 ['gittins_with_my_and_count_rival_drift',
  (0.4324989667265736, -6.3, 25.38129232328409)],
 ['gittins_bb', (0.39709422435313446, 6.6, 24.646297896438725)],
 ['gittins_bb_delta', (0.9625877451708813, 0.5, 33.70830758136635)],
 ['softmax_ucb', (0.00018081513927886215, 35.7, 30.14979270243827)],
 ['multiarmed_bandit_agent', (0.294236542913377, 7.0, 21.104501889407388)],
 ['upper_confidence', (6.578580142975601e-53, 142.1, 29.35114989229553)],
 ['ucb_decay', (0.0005915179478898581, 32.4, 29.823480682173905)],
 ['bayesian_ucb', (0.2697398047344016, -8.8, 25.21428166734083)],
 ['thompson', (0.9482518422547107, -0.4, 19.48948434412773)],
 ['neural', (0.47011626999402834, 7.4, 32.3981480952229

In [27]:
import torch
from torch import nn

In [205]:
class CustomLinearLayer(nn.Module):
    def __init__(self, f_in, f_out, inner=1, list_a=None, list_b=None, list_bias=None):
        super().__init__()
        if list_a is None:
            assert list_b is None and list_bias is None
            list_a = torch.nn.init.xavier_uniform_(torch.empty(f_in, inner))
            list_b = torch.nn.init.xavier_uniform_(torch.empty(f_out, inner))
            list_bias = torch.nn.init.normal_(torch.empty(f_out))
        self.a = self._create_grad_tensor_from_list(list_a).reshape(f_in, inner)
        self.b = self._create_grad_tensor_from_list(list_b).reshape(f_out, inner)
        self.bias = self._create_grad_tensor_from_list(list_bias).reshape(f_out)
        self.a = nn.Parameter(self.a, requires_grad=True)
        self.b = nn.Parameter(self.b, requires_grad=True)
        self.bias = nn.Parameter(self.bias, requires_grad=True)
    def forward(self, x):
        return torch.matmul(x, torch.matmul(self.a, self.b.T)) + self.bias
    def _create_grad_tensor_from_list(self, l):
        a = torch.FloatTensor(l)
        return a

In [203]:
model = nn.Sequential(
    nn.Softsign(),
    CustomLinearLayer(4, 5, 1, [1,2,3,4], [5,6,7,8,9]),
    nn.Sigmoid(),
    nn.Linear(5, 1)
)

TypeError: new(): data must be a sequence (got NoneType)

In [204]:
c = torch.FloatTensor(10,4)
c

tensor([[0.0000e+00, 4.6566e-10, 0.0000e+00, 4.6566e-10],
        [1.9618e-44, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.6302e+20, 6.1943e-04],
        [7.2056e+22, 2.1720e+29, 1.8057e+28, 5.1412e+31],
        [7.5338e+28, 6.1687e+16, 6.3371e-10, 5.4666e+22],
        [8.1600e+17, 2.0190e-19, 1.3563e-19, 4.5071e+16],
        [6.4900e-07, 5.2887e+22, 8.2050e+17, 2.0190e-19],
        [1.3563e-19, 4.5071e+16, 6.3373e-10, 5.2893e+22],
        [8.2050e+17, 2.0190e-19, 1.3563e-19, 4.5071e+16],
        [6.3374e-10, 5.3193e+22, 8.2501e+17, 2.0190e-19]])

In [201]:
torch.sum(model(c)).backward()

In [211]:
list(model[1].a.detach().reshape(-1).numpy())

[1.0, 2.0, 3.0, 4.0]

In [212]:
list(model[1].bias.detach().reshape(-1).numpy())

[1.0, 2.0, 3.0, 4.0, 5.0]

In [198]:
model[1].bias.grad

tensor([-0.6494,  0.2207, -0.3683,  0.4396, -0.0591])

In [104]:
a = torch.FloatTensor([1,2,3,4])
a.requires_grad = True
a

tensor([1., 2., 3., 4.], requires_grad=True)

In [105]:
torch.randn(5)

tensor([-0.1874,  0.0936,  0.6404, -0.6436,  1.3606])

In [96]:
b = torch.tensor.requires_gra(5)
b.requires_grad = True
b

tensor([0.0000e+00, 4.6566e-10, 0.0000e+00, 4.6566e-10, 9.8091e-45],
       requires_grad=True)

In [98]:
a.reshape(-1,1) * b.reshape(1,-1)

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  2.1684e-19,  0.0000e+00,  2.1684e-19,  0.0000e+00],
        [-0.0000e+00, -1.4001e+29, -0.0000e+00, -1.4001e+29, -2.9492e-06],
        [-0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00]],
       grad_fn=<MulBackward0>)

In [62]:
a.reshape(-1,1).shape, b.reshape(1,-1).shape

(torch.Size([4, 1]), torch.Size([1, 5]))

In [4]:
compare(Agent(text=neural), Agent(text=neural), T=1)

/Users/sergmiller/Documents/my/bandits/utils/bandits.py:68: RuntimeWarning: divide by zero encountered in double_scalars
  z = mu_z / sigma * T ** 0.5


(0.0,
 13.0,
 0.0,
 1.0,
 'tmp/b_0.5724676660972491.py',
 'tmp/b_0.9718748026778358.py',
 array([678.]),
 array([665.]))

In [5]:
custom = utils.bandits.neural_with_new_feature_vec.format("{}", "4", "custom_bandits")
compare(Agent(text=custom), Agent(text=neural), T=20)

(0.01720404810730683,
 15.15,
 28.43989275647853,
 0.65,
 'tmp/b_0.5907712293355577.py',
 'tmp/b_0.056487843548509.py',
 array([646., 656., 591., 657., 663., 642., 554., 626., 692., 644., 619.,
        642., 626., 625., 653., 660., 607., 684., 721., 665.]),
 array([654., 623., 632., 619., 672., 586., 596., 598., 650., 625., 608.,
        607., 633., 592., 626., 636., 599., 695., 722., 597.]))

In [7]:
bench(Agent(text=custom))

['gittins', (0.0016901188012321843, 33.1, 33.3360165586712)]
['gittins_with_random', (6.526549088451549e-08, 30.3, 17.731610191970724)]
['gittins_with_count_my', (0.1528315954575815, 11.9, 26.322803802026865)]
['gittins_with_count_rival_drift', (0.004435440838218201, 22.0, 24.44994887520217)]
['gittins_with_my_and_count_rival_drift', (0.2882320284131016, 6.7, 19.95018796903929)]
['gittins_bb', (0.09246776914021125, 20.6, 38.717437931764024)]
['gittins_bb_delta', (0.20056121276838645, -10.6, 26.188547115103578)]
['softmax_ucb', (2.731184720702389e-06, 29.7, 20.025234081028863)]
['multiarmed_bandit_agent', (0.025722625275678677, 14.9, 21.12557691519926)]
['upper_confidence', (6.2371965809041e-29, 135.9, 38.500519477014855)]
['ucb_decay', (6.293470316159448e-08, 31.4, 18.353201355621856)]
['bayesian_ucb', (0.3426252448180823, -10.8, 35.98833144228835)]
['thompson', (0.30570277224145137, 10.5, 32.41681662347492)]
['neural', (0.653871318270467, -4.8, 33.85203095827487)]
<string>:82: Runtime

[['gittins', (0.0016901188012321843, 33.1, 33.3360165586712)],
 ['gittins_with_random', (6.526549088451549e-08, 30.3, 17.731610191970724)],
 ['gittins_with_count_my', (0.1528315954575815, 11.9, 26.322803802026865)],
 ['gittins_with_count_rival_drift',
  (0.004435440838218201, 22.0, 24.44994887520217)],
 ['gittins_with_my_and_count_rival_drift',
  (0.2882320284131016, 6.7, 19.95018796903929)],
 ['gittins_bb', (0.09246776914021125, 20.6, 38.717437931764024)],
 ['gittins_bb_delta', (0.20056121276838645, -10.6, 26.188547115103578)],
 ['softmax_ucb', (2.731184720702389e-06, 29.7, 20.025234081028863)],
 ['multiarmed_bandit_agent', (0.025722625275678677, 14.9, 21.12557691519926)],
 ['upper_confidence', (6.2371965809041e-29, 135.9, 38.500519477014855)],
 ['ucb_decay', (6.293470316159448e-08, 31.4, 18.353201355621856)],
 ['bayesian_ucb', (0.3426252448180823, -10.8, 35.98833144228835)],
 ['thompson', (0.30570277224145137, 10.5, 32.41681662347492)],
 ['neural', (0.653871318270467, -4.8, 33.852030

In [317]:
from importlib import reload
import utils
reload(utils)
reload(utils.bandits)
reload(utils.snowden)
from utils.bandits import exact_gittins

In [24]:
%%time
custom = utils.bandits.neural_with_new_feature_vec.format("{}", "3", "prior_estimations")
compare(Agent(text=custom), Agent(text=utils.bandits.neural), T=20)

CPU times: user 44min 37s, sys: 51.1 s, total: 45min 28s
Wall time: 11min 41s


(0.5639949117876365,
 3.4,
 26.35602397934863,
 0.5,
 'tmp/b_0.8019340940680074.py',
 'tmp/b_0.6023905655594053.py',
 array([557., 633., 619., 681., 585., 600., 658., 698., 684., 603., 648.,
        579., 634., 635., 682., 651., 669., 627., 731., 540.]),
 array([580., 645., 638., 694., 569., 564., 656., 638., 700., 573., 683.,
        602., 663., 636., 638., 626., 655., 605., 717., 564.]))

In [158]:
reload(neural_agent)
from templates import neural as neural_agent

neural_agent_class = neural_agent.NeuralAgent

In [228]:
datasets = snowden.collect_dataset_from_dir(neural_agent_class, 'sessions/', 1/3)

In [229]:
datasets[0].X.shape, datasets[1].X.shape

((1999, 36), (3998, 36))

In [230]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss

In [259]:
import torch
from torch import nn
from torch import optim

In [296]:
def learn(train, val, model_ff, epochs=5, batch_size=64, shuffle=True, freq=10,lr=1e-3):          
    np.random.seed(1)
    ids_nn = np.arange(train.X.shape[0])

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model_ff.parameters(), lr=lr, momentum=0.9)

    time_for_print_loss = lambda i: (i + 1) % freq == 0

    for epoch in np.arange(epochs):
        np.random.shuffle(ids_nn)
        X_train_shuffled = train.X[ids_nn]
        y_train_shuffled = train.y[ids_nn]

        for b in np.arange(0, X_train_shuffled.shape[0], batch_size):
            X_batch = torch.FloatTensor(X_train_shuffled[b:b+batch_size])
            y_batch = torch.FloatTensor(y_train_shuffled[b:b+batch_size])

            optimizer.zero_grad()
            y_pred_logits = model_ff(X_batch).reshape(-1)

            loss = criterion(y_pred_logits, y_batch)
            loss.backward()

            optimizer.step()

            if (b // batch_size + 1) % freq == 0:
                print('train loss in %d epoch in %d batch: %.3f' %
                  (epoch + 1, b // batch_size + 1, loss.item()))

        val_loss = 0
        its = 0
        for b in np.arange(0, val.X.shape[0], batch_size):
            its += 1
            X_batch = torch.FloatTensor(val.X[b:b+batch_size])
            y_batch = torch.FloatTensor(val.y[b:b+batch_size])

            y_pred_logits = model_ff(X_batch).reshape(-1)

            loss = criterion(y_pred_logits, y_batch)
            val_loss += loss.item()
        val_loss /= its
        print('val loss in %d epoch: %.3f' % (epoch + 1, val_loss))

In [251]:
x = [[1,2],[3,4],[5,6]]
np.random.shuffle(x)
print(x)

[[1, 2], [5, 6], [3, 4]]


In [241]:
model = MLPClassifier(hidden_layer_sizes=(64,64),verbose=True, activation='tanh')

In [253]:
datasets[0].X.shape

(1999, 36)

In [236]:
train = resample_eq(datasets[0])
val = resample_eq(datasets[1])

0.5
0.5


In [310]:
INPUT_F = 36
DROP_P = 0.1

model_ff = nn.Sequential(
    nn.LayerNorm(INPUT_F),
    nn.Linear(INPUT_F, 128),
    nn.ReLU(),
    nn.Dropout(DROP_P), 
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Dropout(DROP_P),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Dropout(DROP_P),
    nn.Linear(128, 1)
)

learn(train, val, model_ff, freq=10, batch_size=16,lr=2e-3, epochs=20)

train loss in 1 epoch in 10 batch: 0.690
train loss in 1 epoch in 20 batch: 0.691
train loss in 1 epoch in 30 batch: 0.696
train loss in 1 epoch in 40 batch: 0.685
train loss in 1 epoch in 50 batch: 0.692
train loss in 1 epoch in 60 batch: 0.686
val loss in 1 epoch: 0.687
train loss in 2 epoch in 10 batch: 0.684
train loss in 2 epoch in 20 batch: 0.694
train loss in 2 epoch in 30 batch: 0.687
train loss in 2 epoch in 40 batch: 0.683
train loss in 2 epoch in 50 batch: 0.684
train loss in 2 epoch in 60 batch: 0.673
val loss in 2 epoch: 0.677
train loss in 3 epoch in 10 batch: 0.686
train loss in 3 epoch in 20 batch: 0.692
train loss in 3 epoch in 30 batch: 0.679
train loss in 3 epoch in 40 batch: 0.709
train loss in 3 epoch in 50 batch: 0.675
train loss in 3 epoch in 60 batch: 0.645
val loss in 3 epoch: 0.669
train loss in 4 epoch in 10 batch: 0.695
train loss in 4 epoch in 20 batch: 0.674
train loss in 4 epoch in 30 batch: 0.705
train loss in 4 epoch in 40 batch: 0.667
train loss in 4 e

In [242]:
model.fit(train.X,train.y,)

Iteration 1, loss = 0.68516533
Iteration 2, loss = 0.66485082
Iteration 3, loss = 0.64681494
Iteration 4, loss = 0.64476299
Iteration 5, loss = 0.64279449
Iteration 6, loss = 0.63941776
Iteration 7, loss = 0.63850965
Iteration 8, loss = 0.63727003
Iteration 9, loss = 0.63481114
Iteration 10, loss = 0.63674791
Iteration 11, loss = 0.63222521
Iteration 12, loss = 0.63177047
Iteration 13, loss = 0.63099476
Iteration 14, loss = 0.63040305
Iteration 15, loss = 0.62785761
Iteration 16, loss = 0.62760299
Iteration 17, loss = 0.62897417
Iteration 18, loss = 0.62701603
Iteration 19, loss = 0.62739243
Iteration 20, loss = 0.62802601
Iteration 21, loss = 0.62337626
Iteration 22, loss = 0.62215400
Iteration 23, loss = 0.62217147
Iteration 24, loss = 0.62352573
Iteration 25, loss = 0.61997272
Iteration 26, loss = 0.62083104
Iteration 27, loss = 0.62105614
Iteration 28, loss = 0.61862820
Iteration 29, loss = 0.61771246
Iteration 30, loss = 0.61645791
Iteration 31, loss = 0.61467819
Iteration 32, los

MLPClassifier(activation='tanh', hidden_layer_sizes=(64, 64), verbose=True)

In [243]:
y_pred_train = model.predict_proba(train.X)
y_pred_val = model.predict_proba(val.X)

In [244]:
log_loss(train.y, y_pred_train[:, 1])

0.5746513044617143

In [245]:
log_loss(val.y, y_pred_val[:, 1])

0.7246367046534178

In [ ]:
s = utils.snowden.parse_json_session('/Users/sergmiller/Downloads/7743777.json')

In [318]:
datasets = snowden.collect_dataset_from_dir(neural_agent_class, 'sessions/', 1/3)

sessions/ ['sessions/7743777.json', 'sessions/test_dir/7816604.json', 'sessions/test_dir/7807393.json']
